# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    os.system("wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate")

I = cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(I, 'gray')
plt.show()

In [ ]:
blury = cv2.GaussianBlur(I, (5, 5), 2)
laplasian = cv2.Laplacian(blury, cv2.CV_32F)

def crossing(LoG, thr):
    Y, X = LoG.shape
    result = np.zeros((Y, X))
    for i in range(1, Y-1):
        for j in range(1, X-1):
            block = LoG[(i-1):(i+2),(j-1):(j+2)]
            minimal = np.min(block)
            maximum = np.max(block)

            if minimal * maximum < 0:
                if LoG[i,j] > 0:
                    result[i,j] = LoG[i,j] + np.abs(minimal)
                else:
                    result[i,j] = np.abs(LoG[i,j]) + minimal
    
    result *= 255.0 / result.max()
    result = (result < thr).astype("uint8") * 255
    return cv2.medianBlur(result, 1)


result = crossing(laplasian, 90)

plt.figure(figsize=(10, 7))
plt.imshow(result, 'gray')
plt.show()


## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w~1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj pierwszą część algorytmu detekcji krawędzi Canny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
7. Na jednym obrazie zaznacz piksele należące do obrazu $g_{NH}$ jako czerwone oraz należące do obrazu $g_{NL}$ jako niebieskie.
Wyświetl obraz.

In [ ]:
from math import degrees

def canny(img, th, tl):
    Y, X = img.shape
    filtered = cv2.filter2D(img, -1, np.ones((3,3))/9)
    x = cv2.Sobel(filtered, cv2.CV_64F, 1, 0)
    y = cv2.Sobel(filtered, cv2.CV_64F, 0, 1)
    mag = np.zeros((Y, X))
    phase = np.zeros((Y, X))

    for i in range(Y):
        for j in range(X):
            mag[i,j] = np.sqrt(x[i][j]**2 + y[i][j]**2)
            deg = degrees(np.arctan2(x[i,j], y[i,j]))

            if (0 <= deg < 22.5) or (157.5 <= deg <= 180):
                phase[i,j]=1
            
            elif (22.5 <= deg < 67.5):
                phase[i,j]=2
            
            elif (67.5 <= deg < 112.5):
                phase[i,j]=3
            
            elif (112.5 <= deg < 157.5):
                phase[i,j]=4

    gn = nonmax(phase, mag)
    gnh = gn >= th
    gnl = np.where(np.logical_and(th>gn,gn>=tl), 1, 0)
    return gnh, gnl
    

def nonmax(phase, mag):
    Y, X = phase.shape
    result = np.zeros((Y, X))
    for i in range(1,Y-1):
        for j in range(1, X-1):
            if phase[i,j] == 1:
                if mag[i, j-1] > mag[i, j] or mag[i,j+1] > mag[i,j]:
                    result[i,j] = 0
                else:
                    result[i,j] = mag[i,j]

            elif phase[i,j] == 2:
                if mag[i+1, j-1] > mag[i, j] or mag[i-1,j+1] > mag[i,j]:
                    result[i,j] = 0
                else:
                    result[i,j] = mag[i,j]

            elif phase[i,j] == 3:
                if mag[i+1, j] > mag[i, j] or mag[i-1,j] > mag[i,j]:
                    result[i,j] = 0
                else:
                    result[i,j] = mag[i,j]

            elif phase[i,j] == 4:
                if mag[i-1, j-1] > mag[i, j] or mag[i+1,j+1] > mag[i,j]:
                    result[i,j] = 0
                else:
                    result[i,j] = mag[i,j]
    return result

In [ ]:
I = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

gnh, gnl = canny(I, tl=5, th=10)

plt.imshow(gnh, 'gray')
plt.axis('off')
plt.show()

plt.imshow(gnl, 'gray')
plt.axis('off')
plt.show()


In [ ]:
X, Y = gnh.shape
obraz=np.zeros((X,Y,3))
for i in range (0,X):
    for j in range (0,Y):
        if gnh[i,j] == 1:
            obraz[i,j]=[255,0,0]

        elif gnl[i,j] == 1:
            obraz[i,j]=[0,0,255]

plt.imshow(obraz)
plt.axis('off')
plt.show()

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z wykonaną częściową implementacją w poprzednim ćwiczeniu.
Na czym polegają różnice?

In [ ]:
# Obraz ktory zostal zwrocony przez metode z biblioteki cv2 jest jasniejszy, na niebie jednak pojawiaja sie szare plamy, co w mojej implementacji nie jest widoczne
can = cv2.Canny(I, 5, 10, None, 3, 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
ax1.imshow(can, 'gray')
ax1.axis('off')

ax2.imshow(gnh, 'gray')
ax2.axis('off')
plt.show()